In [1]:
%matplotlib notebook
import sys
from importlib import reload
import pandas as pd
import numpy as np
import csv

from random import randint

reload(sys)

from sklearn.linear_model import LogisticRegression
import xgboost as xgbq
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


from tqdm import tqdm
tqdm.pandas()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
frame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Full_Dataset_Games_Standings_2007-2018.csv', index_col=0)

# Remove games played in 2017-2018 season and use that dataset to test instead
import datetime
    
def selectGamesFromNewSeason(row):
    currentDate = datetime.datetime.strptime(row, '%Y/%m/%d')
    startDate = datetime.datetime(2017, 9, 1)
    endDate = datetime.datetime(2018, 5, 1)
    
    if startDate <= currentDate <= endDate:
        return True
    else: 
        return False

frame.loc[frame['Result'] == 'H', 'Result'] = 1
frame.loc[frame['Result'] == 'A', 'Result'] = 0

newGamesFrame = frame[frame['Date'].apply(lambda row: selectGamesFromNewSeason(row))]
frame = pd.concat([frame, newGamesFrame]).drop_duplicates(keep=False)
display(frame.loc[frame['Date'].str.contains('2018')])

newx_all = newGamesFrame.drop(['Result', 'ID', 'Date', 'HomeTeam', 'AwayTeam'], axis=1)
newy_all = newGamesFrame['Result']

x_all = frame.drop(['Result', 'ID', 'Date', 'HomeTeam', 'AwayTeam'], axis=1)
y_all = frame['Result']

# Standardizing data
from sklearn.preprocessing import scale

columns = x_all.columns.values

for column in columns:
    x_all[column] = scale(x_all[column])
    
newColumns = x_all.columns.values

for newColumn in newColumns:
    newx_all[newColumn] = scale(newx_all[newColumn])
    
display(x_all.head(1))

,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,GF%,PDO,PENDiff,SF%,SDiff,Sh%,Sv,FOW%,W%,Result


,CF%,CSh%,CSv%,FF%,FSh%,FSv%,GDiff,GF%,PDO,PENDiff,SF%,SDiff,Sh%,Sv,FOW%,W%
181,1.090028,-1.650155,-0.408071,0.932755,-1.788723,-0.06546,-0.451797,-0.48737,-1.328975,1.055559,1.025443,1.081616,-2.095765,0.071477,0.361224,-0.126364


In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

In [37]:
# Taken from Siraj Raval
#for measuring training time#for me 
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    
    return f1_score(target, y_pred), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))
    
def train_predictproba(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    display(clf.predict_proba(X_test))
#     print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

In [43]:
clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 43, kernel = 'rbf')
clf_C = xgb.XGBClassifier(seed = 44)
clf_D = SVC(probability=True)

train_predict(clf_A, x_train, y_train, x_test, y_test)
print('')
train_predict(clf_B, x_train, y_train, x_test, y_test)
print('')
train_predict(clf_C, x_train, y_train, x_test, y_test)
print('')
train_predict(clf_D, x_train, y_train, x_test, y_test)
print('')


Training a LogisticRegression using a training set size of 7405. . .
Trained model in 0.0415 seconds
Made predictions in 0.0007 seconds.
0.6631277205648157 0.5715057393652937
F1 score and accuracy score for training set: 0.6631 , 0.5715.
Made predictions in 0.0006 seconds.
F1 score and accuracy score for test set: 0.6547 , 0.5626.

Training a SVC using a training set size of 7405. . .
Trained model in 2.5748 seconds
Made predictions in 1.3094 seconds.
0.6888317035512096 0.5917623227548954
F1 score and accuracy score for training set: 0.6888 , 0.5918.
Made predictions in 0.3237 seconds.
F1 score and accuracy score for test set: 0.6722 , 0.5713.

Training a XGBClassifier using a training set size of 7405. . .
Trained model in 0.6461 seconds
Made predictions in 0.0171 seconds.
0.7086427725756107 0.6344361917623228
F1 score and accuracy score for training set: 0.7086 , 0.6344.
Made predictions in 0.0049 seconds.
F1 score and accuracy score for test set: 0.6569 , 0.5707.

Training a SVC usi

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Trained model in 11.5723 seconds
Made predictions in 1.3025 seconds.
0.6888317035512096 0.5917623227548954
F1 score and accuracy score for training set: 0.6888 , 0.5918.
Made predictions in 0.3213 seconds.
F1 score and accuracy score for test set: 0.6722 , 0.5713.



In [47]:
from sklearn import metrics

predictProba = clf_D.predict_proba(newx_all)
print(clf_D.classes_)
display(predictProba)

newGamesFrame['0_Confidence'] = predictProba[:,0]
newGamesFrame['1_Confidence'] = predictProba[:,1]

predictions = clf_B.predict(newx_all)
print(f1_score(newy_all, predictions), sum(newy_all == predictions) / float(len(predictions)))
# predictDf = pd.DataFrame(predictions, columns=['prediction'])
newGamesFrame['Prediction'] = predictions

newGamesFrame

[0 1]


array([[0.51989135, 0.48010865],
       [0.5137171 , 0.4862829 ],
       [0.42991432, 0.57008568],
       ...,
       [0.39834243, 0.60165757],
       [0.40357018, 0.59642982],
       [0.40460681, 0.59539319]])

0.6947513812154696 0.5930018416206262


,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,W%,Result,Prediction,VegasPrediction,HomePayout,AwayPayout,BetOutcome,VegasBetOutcome,0_Confidence,1_Confidence
11255,https://www.nhl.com/gamecenter/2017020186,2017/11/02,Boston Bruins,Vegas Golden Knights,-1.12,-0.778916,-0.690236,-0.26,-1.321017,-0.902049,...,-0.327273,1,0,1,0.507614,1.770000,-100.000000,50.761421,0.519891,0.480109
11256,https://www.nhl.com/gamecenter/2017020187,2017/11/02,Washington Capitals,New York Islanders,-1.85,-1.019858,-0.441613,-2.09,-1.575384,-0.341264,...,-0.166667,1,0,1,0.751880,1.200000,-100.000000,75.187970,0.513717,0.486283
11257,https://www.nhl.com/gamecenter/2017020188,2017/11/02,Ottawa Senators,Detroit Red Wings,-1.80,1.591117,0.092414,1.01,2.044135,-0.612546,...,-0.044872,1,1,1,0.694444,1.300000,69.444444,69.444444,0.429914,0.570086
11258,https://www.nhl.com/gamecenter/2017020189,2017/11/02,Tampa Bay Lightning,New York Rangers,-0.55,1.602127,1.686500,-0.19,1.966037,2.210855,...,0.461538,0,1,1,0.602410,1.500000,-100.000000,-100.000000,0.400013,0.599987
11259,https://www.nhl.com/gamecenter/2017020190,2017/11/02,Florida Panthers,Columbus Blue Jackets,-4.61,1.404778,-1.736500,-4.64,1.668242,-2.145501,...,-0.303030,0,0,0,0.980392,0.925926,92.592593,92.592593,0.564499,0.435501
11260,https://www.nhl.com/gamecenter/2017020191,2017/11/02,St. Louis Blues,Philadelphia Flyers,-1.48,0.322340,1.266298,-0.79,0.257881,1.725119,...,0.269231,0,1,1,0.540541,1.660000,-100.000000,-100.000000,0.399951,0.600049
11261,https://www.nhl.com/gamecenter/2017020192,2017/11/02,Minnesota Wild,Montréal Canadiens,-7.28,2.199241,1.702287,-6.66,2.698675,2.354117,...,0.066667,1,1,1,0.854701,1.060000,85.470085,85.470085,0.452161,0.547839
11262,https://www.nhl.com/gamecenter/2017020193,2017/11/02,Winnipeg Jets,Dallas Stars,-7.27,1.279969,0.248114,-8.68,1.990895,0.438289,...,-0.037879,1,0,1,0.934579,0.970874,-100.000000,93.457944,0.523466,0.476534
11263,https://www.nhl.com/gamecenter/2017020194,2017/11/02,Colorado Avalanche,Carolina Hurricanes,-8.05,1.788203,0.112443,-6.16,1.828895,0.231109,...,0.145455,1,1,0,1.110000,0.813008,111.000000,-100.000000,0.466653,0.533347
11264,https://www.nhl.com/gamecenter/2017020195,2017/11/02,Calgary Flames,Pittsburgh Penguins,-0.02,-0.918719,1.662550,-1.97,-1.028084,2.492625,...,-0.038462,1,1,1,0.694444,1.300000,69.444444,69.444444,0.389302,0.610698


In [35]:
confusionMatrix = metrics.confusion_matrix(newy_all, predictions)
display(confusionMatrix)

tn = confusionMatrix[0][0]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
tp = confusionMatrix[1][1]
actualYes = fn + tp 
actualNo = tn + fp
predictedYes = fp + tp
predictedNo = tn + fn

print('When home team wins, classifier predicts they will win %6.2f%% of the time' % (tp / actualYes * 100))
print('When home team loses, classifier predicts they will win %6.2f%% of the time' % (fp / actualNo * 100))
print('When home team loses, classifier predicts they will lose %6.2f%% of the time' % (tn / actualNo * 100))
print('When classifer predicts home team will win, home team actually wins %6.2f%% of the time' % (tp / predictedYes * 100))
print('When classifer predicts home team will lose, home team actually loses %6.2f%% of the time' % (tn / predictedNo * 100))

array([[141, 326],
       [116, 503]])

When home team wins, classifier predicts they will win  81.26% of the time
When home team loses, classifier predicts they will win  69.81% of the time
When home team loses, classifier predicts they will lose  30.19% of the time
When classifer predicts home team will win, home team actually wins  60.68% of the time
When classifer predicts home team will lose, home team actually loses  54.86% of the time
1086


In [142]:
oddsFrame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Odds 2007-2018 Raw.csv')

def findPayout(date, homeTeam, awayTeam):
    for index, row in oddsFrame.iterrows():
        if date == row['Date'] and row['HomeTeam'] in homeTeam and row['AwayTeam'] in awayTeam:
            return row['HomeOdds'], row['AwayOdds']

def calculateBetWith100Outcome(prediction, result, homePayout, awayPayout):
    if result == 1 and prediction == 1:
        # Predicted home win correctly
        return homePayout * 100
    elif result == 0 and prediction == 0:
        # Predicted lose correctly
        return awayPayout * 100
    else:
        # Predicted wrong
        return  -100
    
def calculateBetFor100TotalInvestment(prediction, result, homePayout, awayPayout):
    if prediction == 1:
        # Predicted correctly
        return abs(100 / homePayout)
    elif prediction == 0:
        return abs(100 / awayPayout)
    
def calculateBetFor100Outcome(prediction, result, homePayout, awayPayout):
    if result == prediction:
        # Predicted correctly
        return 100
    elif prediction == 0:
        # Predicted lose incorrectly
        return -100 / awayPayout
    elif prediction == 1:
        # Predicted win incorrectly
        return  -100 / homePayout
    
def calculateBetEvenOutcome(prediction, result, homePayout, awayPayout):
    if result == prediction:
    # Predicted home win correctly
        return 100
    else:
    # Predicted wrong
        return  -100
    
def addOdds(row):
    homePayout, awayPayout = findPayout(row['Date'], row['HomeTeam'], row['AwayTeam'])
    vegasPrediction = None
    if homePayout < awayPayout:
        vegasPrediction = 1
    elif homePayout > awayPayout:
        vegasPrediction = 0
    else:
        vegasPrediction = randint(0, 1)
    row['VegasPrediction'] = vegasPrediction
    row['HomePayout'] = homePayout
    row['AwayPayout'] = awayPayout
    
    row['BetEvenOutcome'] = calculateBetEvenOutcome(row['Prediction'], row['Result'], homePayout, awayPayout)
    
    row['BetWith100Outcome'] = calculateBetWith100Outcome(row['Prediction'], row['Result'], homePayout, awayPayout)
    row['BetFor100Outcome'] = calculateBetFor100Outcome(row['Prediction'], row['Result'], homePayout, awayPayout)
    row['BetFor100TotalInvestment'] = calculateBetFor100TotalInvestment(row['Prediction'], row['Result'], homePayout, awayPayout)
    row['VegasBetWith100Outcome'] = calculateBetWith100Outcome(row['VegasPrediction'], row['Result'], homePayout, awayPayout)
    row['VegasBetFor100Outcome'] = calculateBetFor100Outcome(row['VegasPrediction'], row['Result'], homePayout, awayPayout)
    return row
    

newGamesFrame = newGamesFrame.progress_apply(addOdds, axis=1)

display(newGamesFrame)
newGamesFrame.to_csv('2017-2018 Bet Results.csv')



 12%|█▏        | 133/1086 [00:51<10:55,  1.45it/s]

KeyboardInterrupt: 

In [102]:
newGamesFrame[['Prediction', 'Result', 'HomePayout', 'AwayPayout', 'BetFor100Outcome', 'BetFor100TotalInvestment']]

,Prediction,Result,HomePayout,AwayPayout,BetFor100Outcome,BetFor100TotalInvestment
11255,0,1,0.507614,1.770000,-56.497175,56.497175
11256,0,1,0.751880,1.200000,-83.333333,83.333333
11257,1,1,0.694444,1.300000,100.000000,144.000000
11258,1,0,0.602410,1.500000,-166.000000,166.000000
11259,0,0,0.980392,0.925926,100.000000,108.000000
11260,1,0,0.540541,1.660000,-185.000000,185.000000
11261,1,1,0.854701,1.060000,100.000000,117.000000
11262,0,1,0.934579,0.970874,-103.000000,103.000000
11263,1,1,1.110000,0.813008,100.000000,90.090090
11264,1,1,0.694444,1.300000,100.000000,144.000000


In [106]:
print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f (ROI: %6.2f%%)" % (newGamesFrame.shape[0], newGamesFrame['BetFor100Outcome'].sum(), newGamesFrame['BetFor100TotalInvestment'].sum(), (newGamesFrame['BetFor100Outcome'].sum() / newGamesFrame['BetFor100TotalInvestment'].sum() * 100)))
print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['VegasBetFor100Outcome'].sum(), totalGames * 100))
print("If I bet $100 to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['BetEvenOutcome'].sum(), totalGames * 100))

If I bet to make $100 per game (1086 games), I would make $2891.30 on $159857.73 (ROI:   1.81%)
If I bet to make $100 per game (1086 games), I would make $3189.00 on $108600.00
If I bet $100 to make $100 per game (1086 games), I would make $20200.00 on $108600.00


In [55]:
print('Games where Vegas states there is an equal chance of either team winning')
equalOddsGames = newGamesFrame.loc[newGamesFrame['HomePayout'] == newGamesFrame['AwayPayout']]
display(equalOddsGames)

Games where Vegas states there is an equal chance of either team winning


,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,HomePayout,AwayPayout,BetOutcome,VegasBetOutcome,0_Confidence,1_Confidence,BetWith100Outcome,BetFor100Outcome,VegasBetWith100Outcome,VegasBetFor100Outcome
11377,https://www.nhl.com/gamecenter/2017020308,2017/11/19,Vegas Golden Knights,Los Angeles Kings,-0.30,0.863397,-1.314822,-0.22,1.366283,-1.954767,...,0.952381,0.952381,95.238095,95.238095,0.460743,0.539257,95.238095,100.0,-100.000000,-105.0
11439,https://www.nhl.com/gamecenter/2017020370,2017/11/28,Philadelphia Flyers,San Jose Sharks,-1.94,0.652408,-1.257872,-2.92,0.765286,-1.366857,...,0.952381,0.952381,-100.000000,-100.000000,0.491289,0.508711,-100.000000,-105.0,95.238095,100.0
11446,https://www.nhl.com/gamecenter/2017020377,2017/11/29,Boston Bruins,Tampa Bay Lightning,-1.46,-1.248257,-0.749400,-1.39,-1.576881,-1.062136,...,0.952381,0.952381,-100.000000,95.238095,0.535796,0.464204,-100.000000,-105.0,95.238095,100.0
11459,https://www.nhl.com/gamecenter/2017020390,2017/12/01,New York Rangers,Carolina Hurricanes,-5.69,1.424650,0.084943,-4.38,1.604275,0.075353,...,0.952381,0.952381,95.238095,95.238095,0.485471,0.514529,95.238095,100.0,95.238095,100.0
11464,https://www.nhl.com/gamecenter/2017020395,2017/12/01,Colorado Avalanche,New Jersey Devils,0.93,0.154797,-0.381971,0.12,0.543445,-0.578637,...,0.952381,0.952381,-100.000000,-100.000000,0.414947,0.585053,-100.000000,-105.0,95.238095,100.0
11465,https://www.nhl.com/gamecenter/2017020396,2017/12/02,Philadelphia Flyers,Boston Bruins,-0.08,0.059444,-0.073572,-0.25,0.194835,-0.177345,...,0.952381,0.952381,-100.000000,95.238095,0.420551,0.579449,-100.000000,-105.0,95.238095,100.0
11626,https://www.nhl.com/gamecenter/2017020557,2017/12/23,San Jose Sharks,Los Angeles Kings,3.56,-0.787682,-0.539650,3.92,-0.834089,-1.007476,...,0.952381,0.952381,95.238095,-100.000000,0.404366,0.595634,95.238095,100.0,-100.000000,-105.0
11724,https://www.nhl.com/gamecenter/2017020655,2018/01/09,Ottawa Senators,Chicago Blackhawks,-3.97,0.128502,-0.817276,-1.64,-0.115628,-1.423202,...,0.952381,0.952381,95.238095,95.238095,0.521873,0.478127,95.238095,100.0,-100.000000,-105.0
11774,https://www.nhl.com/gamecenter/2017020705,2018/01/19,Florida Panthers,Vegas Golden Knights,-1.91,-0.783326,-0.677722,-2.10,-1.329171,-0.554516,...,0.952381,0.952381,-100.000000,95.238095,0.571704,0.428296,-100.000000,-105.0,-100.000000,-105.0
11853,https://www.nhl.com/gamecenter/2017020784,2018/02/01,Calgary Flames,Tampa Bay Lightning,0.03,-1.372582,-0.108936,-1.01,-1.751073,0.012763,...,0.952381,0.952381,95.238095,95.238095,0.500000,0.500000,95.238095,100.0,95.238095,100.0


In [56]:
totalGames = newGamesFrame.shape[0]
totalGamesWithFavorites = totalGames - equalOddsGames.shape[0]

In [57]:
print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['BetWith100Outcome'].sum(), totalGames * 100))
print("Classifier")
print("     F1 Score: %6.6f" % (f1_score(newy_all, predictions)))
print("     Accuracy: %6.6f   (%d correct / %d games)" % (sum(newy_all == predictions) / float(len(predictions)), sum(newy_all == predictions), len(predictions)))

If I bet $100 per game (1086 games), I would make $1609.04 on $108600.00
Classifier
     F1 Score: 0.694751
     Accuracy: 0.593002   (644 correct / 1086 games)


In [62]:
print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['VegasBetWith100Outcome'].sum(), totalGames * 100))
vegasPredictions = newGamesFrame[['VegasPrediction']].values.flatten()
print("Vegas (Favorites)")
print("     F1 Score: %6.6f" % (f1_score(newy_all, vegasPredictions)))
print("     Accuracy: %6.6f   (%d correct / %d games)" % (sum(newy_all == vegasPredictions) / float(len(vegasPredictions)), sum(newy_all == vegasPredictions), len(vegasPredictions)))

If I bet $100 per game (1086 games), I would make $2420.41 on $108600.00
Vegas (Favorites)
     F1 Score: 0.699717
     Accuracy: 0.609576   (662 correct / 1086 games)


In [14]:
samePrediction = newGamesFrame.loc[newGamesFrame['Prediction'] == newGamesFrame['VegasPrediction']].shape[0]
print("My model is able to identify the favourite team %6.2f%% of the time" % (samePrediction / totalGamesWithFavorites * 100))

My model is able to identify the favourite team  84.84% of the time


In [15]:
print('Assuming Vegas has a pretty good model')
print('     the underdogs win %6.2f%% of the time' % (sum(newy_all != vegasPredictions) / totalGamesWithFavorites * 100))
print('     the favorites win %6.2f%% of the time' % (sum(newy_all == vegasPredictions) / totalGamesWithFavorites * 100))

Assuming Vegas has a pretty good model
     the underdogs win  39.63% of the time
     the favorites win  61.40% of the time


In [110]:
header = ['Team', 'Earnings', 'Total Games', 'Total Investment', 'ROI', 'FavoriteWPercent', 'UnderdogWPercent']
rows = []

def teamPerformance(teamName):
    team = newGamesFrame.loc[(newGamesFrame['HomeTeam'] == teamName) | (newGamesFrame['AwayTeam'] ==  teamName)]
    totalGames = team.shape[0]
    earnings = team['BetFor100Outcome'].sum()
    unequalMatchups = team.loc[team['HomePayout'] != team['AwayPayout']].shape[0]
    favoriteWins = team.loc[team['VegasPrediction'] == team['Result']].shape[0]
#     print("If I bet $100 per game (%d games) on %s, I would make $%6.2f on $%6.2f, %6.2f%% ROI" % (totalGames, teamName, earnings, totalGames * 100, (earnings / totalGames)))
    return [teamName, earnings, totalGames, team['BetFor100TotalInvestment'].sum(), np.float64(earnings / team['BetFor100TotalInvestment'].sum() * 100), np.float64(favoriteWins / unequalMatchups), 1 - (np.float64(favoriteWins / unequalMatchups))]

standings = pd.read_csv(r'/Users/Milton/Developer/Hockey/data/Standings NHL/2018-4-8.csv')
standings.loc[standings['Team'] == 'Arizona Coyotes', 'Team'] = 'Coyotes'

for team in standings[['Team']].values.flatten():
    rows.append(teamPerformance(team))

with open('Bets Team Performance.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(header)
    writer.writerows(rows)
    
bets = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Bets Team Performance.csv')
print('Max ROI: %6.2f%%, Min ROI: %6.2f%%' % (bets['ROI'].max(), bets['ROI'].min()))

Max ROI:  16.06%, Min ROI: -12.82%


0


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


ZeroDivisionError: division by zero

In [97]:
newGamesFrame['ConfidenceDifference'] = abs(newGamesFrame['0_Confidence'] - newGamesFrame['1_Confidence'])
conMax = newGamesFrame['ConfidenceDifference'].max()
conMedian = newGamesFrame['ConfidenceDifference'].median()
conMin = newGamesFrame['ConfidenceDifference'].min()
conMean = newGamesFrame['ConfidenceDifference'].mean()
print('Confidence Difference')
print('     max: %6.2f' % conMax)
print('     min: %6.2f' % conMin)
print('     median: %6.2f' % conMedian)
print('     mean: %6.2f' % conMean)

def calculateAccuracy(threshold):
    confidentFrame = newGamesFrame.loc[newGamesFrame['ConfidenceDifference'] > threshold]
    correctGuesses = confidentFrame.loc[confidentFrame['Result'] == confidentFrame['Prediction']].shape[0]
#     display(confidentFrame[['HomeTeam', 'AwayTeam', 'Prediction', 'VegasPrediction', 'Result', '0_Confidence', '1_Confidence', 'ConfidenceDifference']].sort_values('ConfidenceDifference'))
    print('Threshold: %6.2f, Accuracy: %6.2f%%' % (threshold, (correctGuesses / confidentFrame.shape[0])))
    print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetFor100Outcome'].sum(), totalGames * 100))
    
    samePrediction = confidentFrame.loc[confidentFrame['Prediction'] == confidentFrame['VegasPrediction']].shape[0]
    print("My model is able to identify the favourite team %6.2f%% of the time" % (samePrediction / confidentFrame.shape[0] * 100))

for i in [x / 100 for x in range(0, 28)]:
    calculateAccuracy(i)

# calculateAccuracy(0.20)

# print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetFor100Outcome'].sum(), totalGames * 100))
# print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetWith100Outcome'].sum(), totalGames * 100))
# print("If Vegas bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['VegasBetFor100Outcome'].sum(), totalGames * 100))
# print("If I bet $100 to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetEvenOutcome'].sum(), totalGames * 100))

Confidence Difference
     max:   0.28
     min:   0.00
     median:   0.16
     mean:   0.14
Threshold:   0.00, Accuracy:   0.59%
If I bet to make $100 per game (1059 games), I would make $2632.34 on $108600.00
My model is able to identify the favourite team  85.08% of the time
Threshold:   0.01, Accuracy:   0.59%
If I bet to make $100 per game (1059 games), I would make $2632.34 on $108600.00
My model is able to identify the favourite team  85.08% of the time
Threshold:   0.02, Accuracy:   0.60%
If I bet to make $100 per game (1027 games), I would make $3163.03 on $108600.00
My model is able to identify the favourite team  85.88% of the time
Threshold:   0.03, Accuracy:   0.60%
If I bet to make $100 per game (994 games), I would make $3157.34 on $108600.00
My model is able to identify the favourite team  87.02% of the time
Threshold:   0.04, Accuracy:   0.60%
If I bet to make $100 per game (973 games), I would make $2606.18 on $108600.00
My model is able to identify the favourite tea

In [141]:
header = ['Month', 'Accuracy', 'Earnings', 'Total Games', 'Total Investment', 'ROI', 'FavoriteWPercent', 'UnderdogWPercent']
rows = []

def monthPerformance(monthString):
    date = "/%s/" % monthString
    month = newGamesFrame.loc[newGamesFrame['Date'].str.contains(date)]
    totalGames = month.shape[0]
    earnings = month['BetFor100Outcome'].sum()
    unequalMatchups = month.loc[month['HomePayout'] != month['AwayPayout']].shape[0]
    favoriteWins = month.loc[month['VegasPrediction'] == month['Result']].shape[0]
    accuracy = month.loc[month['Prediction'] == month['Result']].shape[0]
#     print("If I bet $100 per game (%d games) on %s, I would make $%6.2f on $%6.2f, %6.2f%% ROI" % (totalGames, teamName, earnings, totalGames * 100, (earnings / totalGames)))
    return [monthString, np.float64(accuracy / totalGames), earnings, totalGames, month['BetFor100TotalInvestment'].sum(), np.float64(earnings / month['BetFor100TotalInvestment'].sum() * 100), np.float64(favoriteWins / unequalMatchups), 1 - (np.float64(favoriteWins / unequalMatchups))]

months = ['11', '12', '01', '02', '03', '04']
for i in months:
    print(i)
    rows.append(monthPerformance(i))

with open('Bets Month Performance.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(header)
    writer.writerows(rows)

11
12
01
02
03
04
